Approximate run duration: 10 hours

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
cd /content/drive/MyDrive/Work/Sciencing/Google_CoLab_projects/WWD_Model/WWD_pytorch/wwd-model-fitting/final/

/content/drive/MyDrive/Work/Sciencing/Google_CoLab_projects/WWD_Model/WWD_pytorch/wwd-model-fitting/final


In [3]:
import sys
sys.path.append('code/')

In [4]:
from rww_pytorch_model import ParamsJR
from rww_pytorch_model import RNNWWD
from rww_pytorch_model import Model_fitting
import matplotlib.pyplot as plt # for plotting
import numpy as np # for numerical operations
import pandas as pd # for data manipulation
import seaborn as sns # for plotting 
import time # for timer
import os
import torch
from torch.nn.parameter import Parameter

In [5]:
start_time = time.time()

"""Syn_ts_sim ={}
Syn_ts_test ={}
Syn_paras = {}
Syn_sc_mod = {}"""





for i in range(42,50):
    
    print(i)
    sub = 'sub_'+str(i)
    sc_file = '../data/Hagmann/weights.txt'
    ts_file = '../data/Hagmann/syn_ts.txt'
   
    node_size = 66
    mask = np.tril_indices(node_size, -1)
    output_size = 66
    num_epoches = 20
    batch_size = 20
    step_size = .05
    input_size = 2
    tr = .75
    learning_rate = 0.002
    G = 80
    gEE = 6
    gIE = 8.
    gEI = 1


    sc_file = '../data/Hagmann/weights.txt'
    sc = np.loadtxt(sc_file)
    sc = sc -np.diag(np.diag(sc))
    sc = 0.5*(sc.T+sc)
    sc = np.log1p(sc)/np.linalg.norm(np.log1p(sc))

    # sythetic ts
    ts_w = np.loadtxt(ts_file)
    ts = ts_w[:1200]
    fc_emp = np.corrcoef(ts.T)
    
    sc1=sc.copy()
    sc1[sc<0.05]=0.05

    dist = np.nan_to_num(1/sc1)
    dist[sc == 0] = 0
    #SC[SC <SC[SC>0].mean() +1.1*SC[SC>0].std()] =0 # for connectivity 96 only

    par = ParamsJR("WWD", g = [G, 0], g_EE= [gEE, 0], g_IE= [gIE, .25], \
                        g_EI= [gEI, 25], std_in=[0.03, 0],   mu = [100, 0])

    model = RNNWWD(input_size, node_size, batch_size, step_size, tr, sc, dist, False,  par)


    data_mean = [ts.T]
    data_mean =np.array(data_mean)
    # call model fit method
    F = Model_fitting(model, data_mean, num_epoches, 1, learning_rate)

    # fit data(train)

    output_train = F.train()
    gIE_best = F.model.g_IE_m.detach().numpy()#F.output_sim.g_IE[(np.argmax(F.output_sim.fit_corr)+1)*60][0]
    gEI_best = F.model.g_EI_m.detach().numpy()#F.output_sim.g_EI[(np.argmax(F.output_sim.fit_corr)+1)*60][0]
    print([#'g', F.output_sim.g[(np.argmax(F.output_sim.fit_corr)+1)*60][0],\
                            #'gEE', F.output_sim.g_EE[(np.argmax(F.output_sim.fit_corr)+1)*120][0],\
                            'gIE', gIE_best , F.model.g_IE_m.detach().numpy(),\
                            'gEI',  gEI_best, F.model.g_EI_m.detach().numpy()] )
    
    F.model.g_EI = Parameter(torch.tensor(gEI_best , dtype=torch.float32))
    F.model.g_IE = Parameter(torch.tensor(gIE_best, dtype=torch.float32))

    X0 = 0.45 * np.random.uniform(0, 1, (node_size, 6)) + np.array(
                [0, 0, 0, 1.0, 1.0, 1.0])
    hE0 = np.random.uniform(0, 5, (node_size, 50))
    base_batch_num = 20
    
    ts = np.concatenate([ts,ts], axis= 0)
    data_mean = [ts.T]
    data_mean = np.array(data_mean)
    F.ts= data_mean
    output_test = F.test(X0, hE0, base_batch_num)
    
    fc_test = np.corrcoef(F.output_sim.bold_test)
    fc_sim = np.corrcoef(F.output_sim.bold_train)
    fig, ax = plt.subplots(1,4, figsize=(12,8))
    ax[1].imshow(fc_emp-np.diag(np.diag(fc_emp)), cmap='bwr')
    ax[1].set_title('Empirical FC')
    ax[0].plot(F.output_sim.bold_test.T)
    ax[0].set_title('Test')
    ax[2].imshow(fc_test-np.diag(np.diag(fc_test)), cmap='bwr')
    ax[2].set_title('Test FC')
    ax[3].imshow(fc_sim-np.diag(np.diag(fc_sim)), cmap='bwr')
    ax[3].set_title('Train FC')
    plt.show()
    print(np.corrcoef(fc_emp[mask], fc_test[mask])[0,1])

    np.savetxt('../data/Hagmann/bold_test_2p_'+ sub +'.txt', F.output_sim.bold_test)
    np.savetxt('../data/Hagmann/bold_train_2p_'+ sub +'.txt', F.output_sim.bold_train)
    
    gEI = F.output_sim.g_EI[:,0]
    gIE = F.output_sim.g_IE[:,0]
    fig, ax = plt.subplots(1,5)
    ax[0].plot([80])#g)
    ax[1].plot([6])#gEE)
    ax[2].plot(gIE)
    ax[3].plot(gEI)
    ax[4].plot(F.output_sim.loss)
    plt.show()
    fig, ax = plt.subplots(1,6)
    ax[0].plot(F.output_sim.E_test.T)
    ax[1].plot(F.output_sim.I_test.T)
    ax[2].plot(F.output_sim.bold_train.T)
    ax[3].plot(F.output_sim.bold_test.T)
    ax[4].plot(F.output_sim.v_test.T)
    ax[5].plot(F.output_sim.f_test.T)
    
    plt.show()
    
    np.savetxt('../data/Hagmann/parameters_2p_'+ sub +'.txt', np.array([gIE_best, gEI_best]).T)
end_time =  time.time()
print('running time is  {0} \'s'.format(end_time - start_time ))

Output hidden; open in https://colab.research.google.com to view.

In [6]:
print('posterior std: EI', 1/np.sqrt(F.model.g_EI_v.detach().numpy()), 'prior std: EI', 1/np.sqrt(25))

posterior std: EI 0.19118581494955594 prior std: EI 0.2


In [7]:
print('posterior std: IE', 1/np.sqrt(F.model.g_IE_v.detach().numpy()), 'prior std: IE', 1/np.sqrt(0.25))

posterior std: IE 0.5625100291259564 prior std: IE 2.0


In [8]:

Syn_ts_sim ={}
Syn_ts_test ={}
Syn_paras = {}
Syn_sc_mod = {}
for i in range(0, 50):
    sub = 'sub_'+str(i)
    

    Syn_ts_test[sub] = np.loadtxt('../data/Hagmann/bold_test_2p_'+ sub +'.txt')
    Syn_ts_sim[sub] = np.loadtxt('../data/Hagmann/bold_train_2p_'+ sub +'.txt')
    
    
    Syn_paras[sub] = np.loadtxt('../data/Hagmann/parameters_2p_'+ sub +'.txt')
    
np.save('../data/Hagmann/Syn_ts_sim_2p_050.npy', Syn_ts_sim)
np.save('../data/Hagmann/Syn_ts_test_2p_050.npy', Syn_ts_test)

np.save('../data/Hagmann/Syn_fitparas_2p_050.npy', Syn_paras)